In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import time

img = cv2.cvtColor(cv2.imread("./Q3_4.tif"), cv2.COLOR_BGR2GRAY)
H = img.shape[0]
W = img.shape[1]

In [ ]:
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
kernel = 1 / 16 * np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
kernel

In [ ]:
_kernel_size = kernel.shape[0]
assert _kernel_size % 2 == 1  # only handel odd kernel
_s_half = _kernel_size // 2
_l_half = _kernel_size // 2 + 1
padded = np.pad(img, (_s_half, _s_half))
_H = H + _s_half
_W = W + _s_half
padded

In [ ]:
star = time()
box_filtered = np.zeros_like(img)
for i in range(_s_half, _H):
    for j in range(_s_half, _W):
        box_filtered[i - 1, j - 1] = np.sum(kernel * padded[i - _s_half:i + _l_half, j - _s_half:j + _l_half])
end = time()
print('time:', end - star)

In [ ]:
plt.imshow(box_filtered, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
K = 1
sigma = 1
_kernel_size = np.ceil(6 * sigma).astype(int)
if _kernel_size % 2 == 0:
    _kernel_size += 1

# get the distance matrix
R_gen_size = 3 * sigma
R_gen_row = np.arange(1, R_gen_size + 1) ** 2
R_gen = np.zeros((R_gen_size, R_gen_size))
for i in range(1, R_gen_size + 1):
    R_gen[i - 1] = R_gen_row + R_gen_row[i - 1]
R_right = np.vstack((np.flipud(R_gen), R_gen_row, R_gen))
R_mid = np.r_[np.flip(R_gen_row), [0], R_gen_row]
R_left = np.fliplr(R_right)
R = np.hstack((R_left, R_mid.reshape(_kernel_size, 1), R_right))

# generate coefficient w
G = K * np.exp(-R / (2 * sigma ** 2))

kernel = 1 / np.sum(G) * G
# _kernel_size = kernel.shape[0]
assert _kernel_size % 2 == 1  # only handel odd kernel
_s_half = _kernel_size // 2
_l_half = _kernel_size // 2 + 1

padded = np.pad(img, (_s_half, _s_half))
_H = H + _s_half
_W = W + _s_half

In [ ]:
kernel

In [ ]:
star = time()
box_filtered = np.zeros_like(img)
big_mat = np.zeros((H * W, _kernel_size * _kernel_size))
cnt = 0
for i in range(_s_half, _H):
    for j in range(_s_half, _W):
        big_mat[cnt] = padded[i - _s_half:i + _l_half, j - _s_half:j + _l_half].ravel()
        cnt += 1
box_filtered = (big_mat @ kernel.ravel()).reshape(H, W)
end = time()
print('time:', end - star)

In [ ]:
plt.imshow(box_filtered, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
padded

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import time

# img = cv2.cvtColor(cv2.imread("./Q3_4.tif"), cv2.COLOR_BGR2GRAY)
# H = img.shape[0]
# W = img.shape[1]

In [107]:
parallel = 2
arr = np.array([np.arange(9).reshape(3, 3),
                np.arange(1, 10).reshape(3, 3),
                np.arange(2, 11).reshape(3, 3)])
arr

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9]],

       [[ 2,  3,  4],
        [ 5,  6,  7],
        [ 8,  9, 10]]])

In [180]:
def img2big_mat(arr, m_size, pad=True, pad_mode='constant'):
    """
    calculate all the neighbors by iterating the array

    :param pad_mode: refer to definition of `np.pad`, 'edge', 'reflect', 'symmetric' are commonly used pad methods
    :param pad: whether pad the arr after m_size, default is yes
    :param arr: the input image or array
    :param m_size: is the scale of the neighborhood size
    :return: the padded arr, and a (pixel_cnt x m_size**2) size array contains all neighbors
    """
    _H = arr.shape[0]
    _W = arr.shape[1]
    assert _H >= m_size and _W >= m_size
    _half = m_size // 2
    _l_half = _half + 1
    _padded = np.pad(arr, (_half, _half), mode=pad_mode).astype(np.uint8)
    big_mat = np.zeros((_H * _W, m_size * m_size), dtype=np.uint8)
    cnt = 0
    for i in range(_half, _H + 1):
        for j in range(_half, _W + 1):
            big_mat[cnt] = _padded[i - _half:i + _l_half, j - _half:j + _l_half].ravel()
            cnt += 1
    return _padded, big_mat

In [177]:
def hist(arr, _min=0, _max=255, parallel=1):
    """
    Use a tricky method with the efficient numpy. Avoid using for loop twice to iterate the image, because it is slow!

    :param parallel: 0 means that arr is single image or array, otherwise it is an array of multiple images or arrays
    :param arr: the input image or array
    :param _min: minimum value of arr
    :param _max: maximum value of arr
    :return: histogram bins with size of (parallel x _max - _min + 1)
    """
    _hist = np.zeros((parallel, _max - _min + 1))
    if parallel == 1:
        for v in range(_min, _max + 1):
            _hist[v] = np.sum(arr == v)
    else:
        for v in range(_min, _max + 1):
            _hist[:, v] = np.sum(arr == v, axis=1)
    return _hist

In [178]:
img = np.arange(30).reshape(5, 6)
img

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

In [181]:
padded_, arr_big = img2big_mat(img, 3)
input_hist = hist(arr_big, parallel=arr_big.shape[0])
arr_big

array([[ 0,  0,  0,  0,  0,  1,  0,  6,  7],
       [ 0,  0,  0,  0,  1,  2,  6,  7,  8],
       [ 0,  0,  0,  1,  2,  3,  7,  8,  9],
       [ 0,  0,  0,  2,  3,  4,  8,  9, 10],
       [ 0,  0,  0,  3,  4,  5,  9, 10, 11],
       [ 0,  0,  0,  4,  5,  0, 10, 11,  0],
       [ 0,  0,  1,  0,  6,  7,  0, 12, 13],
       [ 0,  1,  2,  6,  7,  8, 12, 13, 14],
       [ 1,  2,  3,  7,  8,  9, 13, 14, 15],
       [ 2,  3,  4,  8,  9, 10, 14, 15, 16],
       [ 3,  4,  5,  9, 10, 11, 15, 16, 17],
       [ 4,  5,  0, 10, 11,  0, 16, 17,  0],
       [ 0,  6,  7,  0, 12, 13,  0, 18, 19],
       [ 6,  7,  8, 12, 13, 14, 18, 19, 20],
       [ 7,  8,  9, 13, 14, 15, 19, 20, 21],
       [ 8,  9, 10, 14, 15, 16, 20, 21, 22],
       [ 9, 10, 11, 15, 16, 17, 21, 22, 23],
       [10, 11,  0, 16, 17,  0, 22, 23,  0],
       [ 0, 12, 13,  0, 18, 19,  0, 24, 25],
       [12, 13, 14, 18, 19, 20, 24, 25, 26],
       [13, 14, 15, 19, 20, 21, 25, 26, 27],
       [14, 15, 16, 20, 21, 22, 26, 27, 28],
       [15

In [203]:
_m_size = arr_big.shape[1]
_half = _m_size // 2
r_pdf = input_hist / _m_size
r_cdf = np.round(255 * np.cumsum(r_pdf, axis=1)).astype(int)
output_image = np.zeros(arr_big.shape[0])
# for i in range(256):
#     output_image[:, arr_big == i] = r_cdf[:, i]

In [212]:
center_pixel = arr_big[:, _m_size // 2]
for i in range(arr_big.shape[0]):
    output_image[i] = r_cdf[i, center_pixel[i]]

In [206]:
arr_big

array([[ 0,  0,  0,  0,  0,  1,  0,  6,  7],
       [ 0,  0,  0,  0,  1,  2,  6,  7,  8],
       [ 0,  0,  0,  1,  2,  3,  7,  8,  9],
       [ 0,  0,  0,  2,  3,  4,  8,  9, 10],
       [ 0,  0,  0,  3,  4,  5,  9, 10, 11],
       [ 0,  0,  0,  4,  5,  0, 10, 11,  0],
       [ 0,  0,  1,  0,  6,  7,  0, 12, 13],
       [ 0,  1,  2,  6,  7,  8, 12, 13, 14],
       [ 1,  2,  3,  7,  8,  9, 13, 14, 15],
       [ 2,  3,  4,  8,  9, 10, 14, 15, 16],
       [ 3,  4,  5,  9, 10, 11, 15, 16, 17],
       [ 4,  5,  0, 10, 11,  0, 16, 17,  0],
       [ 0,  6,  7,  0, 12, 13,  0, 18, 19],
       [ 6,  7,  8, 12, 13, 14, 18, 19, 20],
       [ 7,  8,  9, 13, 14, 15, 19, 20, 21],
       [ 8,  9, 10, 14, 15, 16, 20, 21, 22],
       [ 9, 10, 11, 15, 16, 17, 21, 22, 23],
       [10, 11,  0, 16, 17,  0, 22, 23,  0],
       [ 0, 12, 13,  0, 18, 19,  0, 24, 25],
       [12, 13, 14, 18, 19, 20, 24, 25, 26],
       [13, 14, 15, 19, 20, 21, 25, 26, 27],
       [14, 15, 16, 20, 21, 22, 26, 27, 28],
       [15

In [205]:
r_cdf

array([[170, 198, 198, ..., 255, 255, 255],
       [113, 142, 170, ..., 255, 255, 255],
       [ 85, 113, 142, ..., 255, 255, 255],
       ...,
       [ 85,  85,  85, ..., 255, 255, 255],
       [ 85,  85,  85, ..., 255, 255, 255],
       [142, 142, 142, ..., 255, 255, 255]])

In [208]:
center_pixel

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=uint8)

In [211]:
r_cdf[0, center_pixel[0]]

170

In [3]:
arr = cv2.cvtColor(cv2.imread('Q3_3.tif'), cv2.COLOR_RGB2GRAY)
m_size = 5
_H = arr.shape[0]
_W = arr.shape[1]
assert _H >= m_size and _W >= m_size
_half = m_size // 2
_l_half = _half + 1
_padded = np.pad(arr, (_half, _half))
big_mat = np.zeros((_H * _W, m_size * m_size))
cnt = 0
for i in range(_half, _H + _half):
    for j in range(_half, _W + _half):
        big_mat[cnt] = _padded[i - _half:i + _l_half, j - _half:j + _l_half].ravel()
        cnt += 1

In [9]:
_padded

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0, 227, ..., 227,   0,   0],
       ...,
       [  0,   0, 227, ..., 227,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [21]:
big_mat[510].reshape(5, 5)

array([[  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [227., 227., 227., 227.,   0.],
       [227., 227., 227., 227.,   0.],
       [227., 227., 227., 227.,   0.]])